** valencia bounding box  
{"type":"Polygon","coordinates":[[[-0.124146,38.953446],[-0.547119,38.953446],[-0.547119,39.481696],[-0.124146,39.481696],[-0.124146,38.953446]]]}  

** data   
pre-event = 2024-10-07  
post-event = 2024-10-31  

** Data download from ASF search portal: parameters  
    * GRD-HD, ascending, instrument IW, VV+VH pols

In [ ]:
# Importing functions from package--
from pysarflow import read_grd_product, subset_AOI, apply_orbit_file, thermal_noise_removal, border_noise_removal, \
    radiometric_calibration, speckle_filter,terrain_correction, stack, band_difference, plotBand, conversion_to_db,export,\
    maskPermanentWater,generateFloodMask, convert_0_to_nan

Read grd file

In [ ]:
# Path to the .SAFE directory
event_product_path = "../data/S1A_IW_GRDH_1SDV_20241031T180256_20241031T180321_056350_06E71E_479F.SAFE"
event_product=read_grd_product(event_product_path)
# plotBand(pre_event_product, "Intensity_VV", 
#          post_event_product, "Intensity_VV", 
#          vmin=0, vmax=100000, figsize=(6,4))

Subsetting

In [ ]:
aoi_bbox=[-0.547119, 38.953446,-0.124146,39.481696]
product_subset=subset_AOI(product=event_product, bbox=aoi_bbox)
# plotBand(pre_event_product_subset, "Intensity_VV", 
#          post_event_product_subset, "Intensity_VV", 
#          vmin=0, vmax=10000, figsize=(6,4))

Apply orbit file to subset

In [ ]:
# pre_event_product_orbit = apply_orbit_file(pre_event_product_subset)
product_orbit = apply_orbit_file(product_subset)

Thermal noise removal

In [ ]:
product_tnr = thermal_noise_removal(product_orbit)

Border noise removal

In [ ]:
product_bnr = border_noise_removal(product_tnr)

Radiometric callibration

In [ ]:
product_calibrated = radiometric_calibration(product_bnr, polarization='VV', pols_selected='VV')
# plotBand(pre_event_product_calibrated, "Sigma0_VV", 
#          post_event_product_calibrated, "Sigma0_VV", 
#          vmin=0, vmax=0.1, figsize=(6,4))

Speckle Filtering

In [ ]:
product_filtered = speckle_filter(product_calibrated)
# plotBand(pre_event_product_filtered, "Sigma0_VV_db", 
#          post_event_product_filtered, "Sigma0_VV_db", 
#          vmin=0, vmax=1,figsize=(6,4))


### Terrain correction

In [ ]:
product_tc = terrain_correction(product_filtered,  sourceBands='Sigma0_VV')

# product_tc_updated=convert_0_to_nan(product_tc)
# plotBand(pre_event_product_tc, "Sigma0_VV_db", 
#          post_event_product_tc, "Sigma0_VV_db", 
#          vmin=0, vmax=0.1,figsize=(6,4))

Identifying known permanent water areas

In [ ]:
product_perm_water= maskPermanentWater(product_tc)

In [ ]:
product_flood_mapping = generateFloodMask(product_perm_water, threshold=0.04)
# plotBand(binary_flood, 'flooded', vmin=0, vmax=1, figsize=(6,4))

In [ ]:
export(product_flood_mapping,"./binary_floodinggg.tif")